In [1]:
import sys, os
sys.path.append('/home/A00512318/TCN')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from TCN.mnist_pixel.model import TCN
import numpy as np

In [2]:
from torchvision import datasets, transforms

def data_generator(root, batch_size):
    train_set = datasets.FashionMNIST(root=root, train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))
    test_set = datasets.FashionMNIST(root=root, train=False, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ]))

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
    return train_loader, test_loader

In [3]:
def trainTCN(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda: 
            data, target = data.to(device), target.to(device)
        data = data.view(-1, input_channels, seq_length)
        if permutee:
            data = data[:, :, permute]
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        if parameters['clip'] > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), parameters['clip'])
        optimizer.step()
        train_loss += loss.item()
        train_losses_.append(train_loss)
        steps += seq_length
        if batch_idx > 0 and batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * parameters['batch_size'], len(train_loader.dataset),
                100. * batch_idx / len(train_loader), train_loss/log_interval, steps))
            train_loss = 0  

In [4]:
def testTCN():
    test_loss = 0
    correct = 0
    correct_class = list(0. for i in range(10))
    correct_total = list(0. for i in range(10))
    tot = 0
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.to(device), target.to(device)
            data = data.view(-1, input_channels, seq_length)
            if permutee:
                data = data[:, :, permute]
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            _, pred = torch.max(output, 1)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
#             print(len(target.data.view_as(pred)))
            c = (pred == target).squeeze()
            tot += 1
#             if tot != 313:
#             for i in range(len(test_loader.dataset) // batch_size):
# #                     print(pred[i], target.data.view_as(pred)[i])
#                 print(i)
#                 label = pred[i]
#                 if (pred[i] == target.data.view_as(pred)[i]):
#                     correct_class[label] += c[i].item()
#                 correct_total[label] += 1
                    
                
#     print(tot)
#     for i in range(10):
#         print('Accuracy of %5s : %2d %%' % (
#             classes[i], 100 * correct_class[i] / correct_total[i]))

    test_loss /= len(test_loader.dataset)
#     print(correct.item())
    accuracies_.append(correct.item() / 10000.)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Print model's state_dict
def models_state_dict(model):
    print("Model's state_dict:")
    for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
def optimizers_state_dict():
    print("Optimizer's state_dict:")
    for var_name in optimizer.state_dict():
        print(var_name, "\t", optimizer.state_dict()[var_name])

In [6]:
import pickle

def save_list_params(obj):
    with open('list_params.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_list_params():
    with open('list_params.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
import random
num_models = 50
seen_hyper = set()
list_parameters = [{} for i in range(0, num_models)]
for i in range(0, num_models):
    while(True):
        list_parameters[i]['batch_size'] = random.choice([16, 32, 64, 128])
        list_parameters[i]['dropout'] = np.random.uniform(0., 1)
        list_parameters[i]['clip'] = np.random.uniform(0.3, 1)
        list_parameters[i]['lr'] = pow(10, np.random.uniform(-6, 1))
        list_parameters[i]['ksize'] = np.random.randint(2, 10)
        list_parameters[i]['levels'] = np.random.randint(2, 11)
        list_parameters[i]['optim'] = 'Adam'
        list_parameters[i]['nhid'] = np.random.randint(15, 30)
        list_parameters[i]['epochs'] = np.random.randint(1, 6)
        k = "".join(str(v) + '_' for k, v in list_parameters[i].items())
        if k not in seen_hyper:
            seen_hyper.add(k)
            break
save_list_params(list_parameters)
[param for param in list_parameters]

[{'batch_size': 32,
  'dropout': 0.1934688879645705,
  'clip': 0.5143058097996211,
  'lr': 1.8394982956044217e-05,
  'ksize': 5,
  'levels': 6,
  'optim': 'Adam',
  'nhid': 21,
  'epochs': 5},
 {'batch_size': 128,
  'dropout': 0.3203992070215431,
  'clip': 0.8603532372826708,
  'lr': 0.00932323004931024,
  'ksize': 5,
  'levels': 5,
  'optim': 'Adam',
  'nhid': 27,
  'epochs': 1}]

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_channels = 1 # just one for the image
classes = ('T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')
n_classes = len(classes) # 10 classes for fashion-mnist
log_interval = 100
seed = 1111
torch.manual_seed(seed)
permutee = False
input_channels = 1
seq_length = int(784 / input_channels)

cuda = True
steps = 0
for model_indx, parameters in enumerate(list_parameters):
    print(parameters)
    steps = 0
#     permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
#     permute = permute.to(device)
    
    train_loader, test_loader = data_generator('../data/fashion_mnist', parameters['batch_size'])
    
    model = TCN(input_channels, n_classes, hidden_units=parameters['nhid'], levels=parameters['levels'], kernel_size=parameters['ksize'], dropout=parameters['dropout'])
    model = nn.DataParallel(model)
    model.to(device)

    print(count_parameters(model))
    lr = parameters['lr']
    optimizer = getattr(optim, parameters['optim'])(model.parameters(), lr=lr)
    accuracies_ = []
    train_losses_ = []
    test_losses_ = []
    file_name = 'model_{0}.pt'.format(model_indx)
   
    for epoch in range(1, parameters['epochs']+1):
        trainTCN(epoch)
        testTCN()
        if epoch % 5 == 0: 
            print('Saving checkpoint for model.....')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.module.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_losses': train_losses_,
                'test_losses': test_losses_,
                'accuracies': accuracies_,
                'curr_lr': lr,
            }, file_name)
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
    torch.save({
        'model_state_dict': model.module.state_dict(),
        'accuracies': accuracies_,
        'train_losses': train_losses_,
        'test_losses': test_losses_,
        'optimizer_state_dict': optimizer.state_dict(),
    }, file_name)
    print('Saved as %s' % file_name)    

{'batch_size': 32, 'dropout': 0.1934688879645705, 'clip': 0.5143058097996211, 'lr': 1.8394982956044217e-05, 'ksize': 5, 'levels': 6, 'optim': 'Adam', 'nhid': 21, 'epochs': 5}
25126
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.428643	Steps: 79184
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.382648	Steps: 157584
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.371411	Steps: 235984
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.350071	Steps: 314384
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.347870	Steps: 392784
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.334228	Steps: 471184
Train Epoch: 1 [22400/60000 (37%)]	Loss: 2.323529	Steps: 549584
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.312629	Steps: 627984
Train Epoch: 1 [28800/60000 (48%)]	Loss: 2.311389	Steps: 706384
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.297698	Steps: 784784
Train Epoch: 1 [35200/60000 (59%)]	Loss: 2.290565	Steps: 863184
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.285185	Steps: 941584
Train Epoch: 1 [41600/60000 (69%)]	Loss: 2.264798	Steps:

/home/A00512318/anaconda3/envs/tcn/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 2.1628, Accuracy: 1948/10000 (19%)

Train Epoch: 2 [3200/60000 (5%)]	Loss: 2.176939	Steps: 1549184
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.134580	Steps: 1627584
Train Epoch: 2 [9600/60000 (16%)]	Loss: 2.117273	Steps: 1705984
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.091464	Steps: 1784384
Train Epoch: 2 [16000/60000 (27%)]	Loss: 2.062406	Steps: 1862784
Train Epoch: 2 [19200/60000 (32%)]	Loss: 2.052197	Steps: 1941184
Train Epoch: 2 [22400/60000 (37%)]	Loss: 2.025826	Steps: 2019584
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.999580	Steps: 2097984
Train Epoch: 2 [28800/60000 (48%)]	Loss: 1.982327	Steps: 2176384
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.955436	Steps: 2254784
Train Epoch: 2 [35200/60000 (59%)]	Loss: 1.946864	Steps: 2333184
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.922619	Steps: 2411584
Train Epoch: 2 [41600/60000 (69%)]	Loss: 1.907333	Steps: 2489984
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.880842	Steps: 2568384
Train Epoch: 2 [48000/60000 (80%)

KeyboardInterrupt: 

In [11]:
mmodel = TCN()
mmodel.load_state_dict(torch.load('model_0.pt')['model_state_dict'])
model.train()

AttributeError: 'TCN' object has no attribute 'module'